In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
import albumentations as A
from albumentations.core.composition import OneOf
import cv2

# Parameters
img_size = (256, 256)
batch_size = 16

# ✅ Dice coefficient
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    y_true_f = tf.reshape(y_true, [tf.shape(y_true)[0], -1])
    y_pred_f = tf.reshape(y_pred, [tf.shape(y_pred)[0], -1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=1)
    union = tf.reduce_sum(y_true_f, axis=1) + tf.reduce_sum(y_pred_f, axis=1)
    dice = tf.reduce_mean((2. * intersection + smooth) / (union + smooth))
    return dice

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coefficient(y_true, y_pred)

def combined_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    dsc = dice_loss(y_true, y_pred)
    return bce + dsc

# ✅ Augmentations
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=15, p=0.3),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=0, p=0.3)
])


class DataGen(Sequence):
    def __init__(self, images, masks, batch_size, augment=False):
        self.images = images
        self.masks = masks
        self.batch_size = batch_size
        self.augment = augment

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.images[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.masks[idx * self.batch_size:(idx + 1) * self.batch_size]

        imgs = []
        msks = []

        for img_path, mask_path in zip(batch_x, batch_y):
            img = cv2.imread(img_path)
            mask = cv2.imread(mask_path, 0)

            img = cv2.resize(img, img_size)
            mask = cv2.resize(mask, img_size)

            if self.augment:
                augmented = transform(image=img, mask=mask)
                img = augmented['image']
                mask = augmented['mask']

            img = img.astype(np.float32) / 255.0
            mask = (mask > 127).astype(np.float32)
            mask = np.expand_dims(mask, axis=-1)

            imgs.append(img)
            msks.append(mask)

        return np.array(imgs), np.array(msks)

# ✅ Build  U-Net
def get_unet(input_shape):
    inputs = layers.Input(input_shape)

    def conv_block(x, filters):
        x = layers.Conv2D(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        x = layers.Conv2D(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        return layers.ReLU()(x)

    def encoder_block(x, filters):
        f = conv_block(x, filters)
        p = layers.MaxPooling2D()(f)
        return f, p

    def decoder_block(x, skip, filters):
        x = layers.UpSampling2D()(x)
        x = layers.Concatenate()([x, skip])
        return conv_block(x, filters)

    f1, p1 = encoder_block(inputs, 32)
    f2, p2 = encoder_block(p1, 64)
    f3, p3 = encoder_block(p2, 128)

    bottleneck = conv_block(p3, 256)

    d3 = decoder_block(bottleneck, f3, 128)
    d2 = decoder_block(d3, f2, 64)
    d1 = decoder_block(d2, f1, 32)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(d1)

    return keras.Model(inputs, outputs)

# ✅ Get image/mask pairs
def get_image_mask_pairs(image_dir, mask_dir):
    image_paths = []
    mask_paths = []

    for fname in os.listdir(image_dir):
        name, _ = os.path.splitext(fname)
        mask_file = name + ".png"
        img_path = os.path.join(image_dir, fname)
        mask_path = os.path.join(mask_dir, mask_file)

        if os.path.exists(mask_path):
            image_paths.append(img_path)
            mask_paths.append(mask_path)
        else:
            print(f"[!] No mask for {fname}, skipping.")

    print(f"\n✅ Found {len(image_paths)} valid image-mask pairs.\n")
    return image_paths, mask_paths

# ✅ Paths to your data
image_dir = "/content/drive/MyDrive/fundus images"
mask_dir = "/content/drive/MyDrive/MASK"

# ✅ Load and split data
image_paths, mask_paths = get_image_mask_pairs(image_dir, mask_dir)
train_imgs, val_imgs, train_masks, val_masks = train_test_split(image_paths, mask_paths, test_size=0.2, random_state=42)

train_gen = DataGen(train_imgs, train_masks, batch_size=batch_size, augment=True)
val_gen = DataGen(val_imgs, val_masks, batch_size=batch_size)

# ✅ Build and compile model
model = get_unet((*img_size, 3))
model.compile(optimizer='adam', loss=combined_loss, metrics=['accuracy', dice_coefficient])

# ✅ Callbacks
checkpoint = keras.callbacks.ModelCheckpoint("best_model.h5", monitor='val_dice_coefficient',
                                              mode='max', save_best_only=True, verbose=1)

# ✅ Train
model.fit(train_gen, validation_data=val_gen, epochs=50, callbacks=[checkpoint])


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


[!] No mask for G1020-820.png, skipping.
[!] No mask for G1020-892 (1).png, skipping.
[!] No mask for G1020-893 (1).png, skipping.

✅ Found 1529 valid image-mask pairs.



/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 623ms/step - accuracy: 0.8219 - dice_coefficient: 0.2215 - loss: 1.2925
Epoch 1: val_dice_coefficient improved from -inf to 0.02027, saving model to best_model.h5


77/77 ━━━━━━━━━━━━━━━━━━━━ 99s 781ms/step - accuracy: 0.8235 - dice_coefficient: 0.2224 - loss: 1.2896 - val_accuracy: 0.0904 - val_dice_coefficient: 0.0203 - val_loss: 7.1595
Epoch 2/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step - accuracy: 0.9895 - dice_coefficient: 0.5663 - loss: 0.6174
Epoch 2: val_dice_coefficient did not improve from 0.02027
77/77 ━━━━━━━━━━━━━━━━━━━━ 30s 384ms/step - accuracy: 0.9895 - dice_coefficient: 0.5669 - loss: 0.6165 - val_accuracy: 0.9837 - val_dice_coefficient: 0.0039 - val_loss: 1.2727
Epoch 3/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step - accuracy: 0.9920 - dice_coefficient: 0.6932 - loss: 0.4081
Epoch 3: val_dice_coefficient did not improve from 0.02027
77/77 ━━━━━━━━━━━━━━━━━━━━ 33s 430ms/step - accuracy: 0.9920 - dice_coefficient: 0.6934 - loss: 0.4078 - val_accuracy: 0.9837 - val_dice_coefficient: 0.0024 - val_loss: 1.1111
Epoch 4/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step - accuracy: 0.9937 - dice_coefficient: 0.7673 - loss: 0.2940
Epoch 4: val_di

...

77/77 ━━━━━━━━━━━━━━━━━━━━ 30s 390ms/step - accuracy: 0.9978 - dice_coefficient: 0.9297 - loss: 0.0774 - val_accuracy: 0.9976 - val_dice_coefficient: 0.9202 - val_loss: 0.0897
Epoch 19/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step - accuracy: 0.9978 - dice_coefficient: 0.9277 - loss: 0.0792
Epoch 19: val_dice_coefficient did not improve from 0.92023
77/77 ━━━━━━━━━━━━━━━━━━━━ 45s 443ms/step - accuracy: 0.9978 - dice_coefficient: 0.9277 - loss: 0.0792 - val_accuracy: 0.9957 - val_dice_coefficient: 0.8141 - val_loss: 0.2050
Epoch 20/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step - accuracy: 0.9979 - dice_coefficient: 0.9307 - loss: 0.0757
Epoch 20: val_dice_coefficient did not improve from 0.92023
77/77 ━━━━━━━━━━━━━━━━━━━━ 30s 385ms/step - accuracy: 0.9979 - dice_coefficient: 0.9306 - loss: 0.0758 - val_accuracy: 0.9970 - val_dice_coefficient: 0.8800 - val_loss: 0.1323
Epoch 21/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step - accuracy: 0.9977 - dice_coefficient: 0.9251 - loss: 0.0821
Epoch 21: 

77/77 ━━━━━━━━━━━━━━━━━━━━ 30s 381ms/step - accuracy: 0.9981 - dice_coefficient: 0.9383 - loss: 0.0671 - val_accuracy: 0.9978 - val_dice_coefficient: 0.9245 - val_loss: 0.0844
Epoch 28/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step - accuracy: 0.9982 - dice_coefficient: 0.9424 - loss: 0.0625
Epoch 28: val_dice_coefficient did not improve from 0.92447
77/77 ━━━━━━━━━━━━━━━━━━━━ 29s 380ms/step - accuracy: 0.9982 - dice_coefficient: 0.9424 - loss: 0.0625 - val_accuracy: 0.9975 - val_dice_coefficient: 0.9121 - val_loss: 0.0973
Epoch 29/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step - accuracy: 0.9982 - dice_coefficient: 0.9406 - loss: 0.0644
Epoch 29: val_dice_coefficient improved from 0.92447 to 0.92796, saving model to best_model.h5


77/77 ━━━━━━━━━━━━━━━━━━━━ 35s 447ms/step - accuracy: 0.9982 - dice_coefficient: 0.9406 - loss: 0.0644 - val_accuracy: 0.9977 - val_dice_coefficient: 0.9280 - val_loss: 0.0806
Epoch 30/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step - accuracy: 0.9981 - dice_coefficient: 0.9404 - loss: 0.0647
Epoch 30: val_dice_coefficient did not improve from 0.92796
77/77 ━━━━━━━━━━━━━━━━━━━━ 29s 379ms/step - accuracy: 0.9981 - dice_coefficient: 0.9404 - loss: 0.0648 - val_accuracy: 0.9977 - val_dice_coefficient: 0.9255 - val_loss: 0.0828
Epoch 31/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step - accuracy: 0.9982 - dice_coefficient: 0.9426 - loss: 0.0622
Epoch 31: val_dice_coefficient did not improve from 0.92796
77/77 ━━━━━━━━━━━━━━━━━━━━ 29s 378ms/step - accuracy: 0.9982 - dice_coefficient: 0.9426 - loss: 0.0623 - val_accuracy: 0.9976 - val_dice_coefficient: 0.9216 - val_loss: 0.0862
Epoch 32/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step - accuracy: 0.9980 - dice_coefficient: 0.9376 - loss: 0.0675
Epoch 32: 

77/77 ━━━━━━━━━━━━━━━━━━━━ 30s 382ms/step - accuracy: 0.9982 - dice_coefficient: 0.9418 - loss: 0.0630 - val_accuracy: 0.9979 - val_dice_coefficient: 0.9282 - val_loss: 0.0797
Epoch 41/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step - accuracy: 0.9982 - dice_coefficient: 0.9426 - loss: 0.0619
Epoch 41: val_dice_coefficient improved from 0.92820 to 0.92872, saving model to best_model.h5


77/77 ━━━━━━━━━━━━━━━━━━━━ 30s 389ms/step - accuracy: 0.9982 - dice_coefficient: 0.9426 - loss: 0.0619 - val_accuracy: 0.9980 - val_dice_coefficient: 0.9287 - val_loss: 0.0791
Epoch 42/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step - accuracy: 0.9983 - dice_coefficient: 0.9475 - loss: 0.0568
Epoch 42: val_dice_coefficient did not improve from 0.92872
77/77 ━━━━━━━━━━━━━━━━━━━━ 30s 381ms/step - accuracy: 0.9983 - dice_coefficient: 0.9475 - loss: 0.0568 - val_accuracy: 0.9979 - val_dice_coefficient: 0.9282 - val_loss: 0.0800
Epoch 43/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step - accuracy: 0.9984 - dice_coefficient: 0.9510 - loss: 0.0532
Epoch 43: val_dice_coefficient did not improve from 0.92872
77/77 ━━━━━━━━━━━━━━━━━━━━ 29s 379ms/step - accuracy: 0.9984 - dice_coefficient: 0.9510 - loss: 0.0532 - val_accuracy: 0.9970 - val_dice_coefficient: 0.8812 - val_loss: 0.1318
Epoch 44/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step - accuracy: 0.9983 - dice_coefficient: 0.9484 - loss: 0.0558
Epoch 44: 

In [ ]:
import numpy as np
from tqdm import tqdm

def dice_score_numpy(y_true, y_pred, smooth=1e-6):
    y_true_f = y_true.reshape(-1)
    y_pred_f = y_pred.reshape(-1)
    intersection = np.sum(y_true_f * y_pred_f)
    union = np.sum(y_true_f) + np.sum(y_pred_f)
    return (2. * intersection + smooth) / (union + smooth)

val_preds = []
val_trues = []

for i in tqdm(range(len(val_gen)), desc="Evaluating"):
    x_batch, y_batch = val_gen[i]
    if x_batch.shape[0] == 0:
        continue
    preds = model.predict(x_batch)
    preds = (preds > 0.5).astype(np.float32)

    val_preds.append(preds)
    val_trues.append(y_batch)

val_preds = np.vstack(val_preds)
val_trues = np.vstack(val_trues)

dice = dice_score_numpy(val_trues, val_preds)
print(f"\n✅ Final Dice Coefficient on Validation Set: {dice:.4f}")


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step


Evaluating:   5%|▌         | 1/20 [00:00<00:08,  2.24it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step


Evaluating:  10%|█         | 2/20 [00:00<00:08,  2.22it/s]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step


Evaluating:  95%|█████████▌| 19/20 [00:08<00:00,  2.26it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


Evaluating: 100%|██████████| 20/20 [00:08<00:00,  2.27it/s]



✅ Final Dice Coefficient on Validation Set: 0.9170


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
